<a href="https://colab.research.google.com/github/salahhh0807/Capstone/blob/main/De_ID_RNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Set-up

In [ ]:
# mount to desired path
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# install libraries
!pip install elmoformanylangs==0.0.2
!pip uninstall overrides
!pip install overrides==3.1.0
!pip install seqeval==0.0.12

Found existing installation: overrides 3.1.0
Uninstalling overrides-3.1.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/overrides-3.1.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/overrides/*
Proceed (Y/n)? Y
  Successfully uninstalled overrides-3.1.0
  Using cached overrides-3.1.0-py3-none-any.whl


In [ ]:
# import libraries
import os
import re
import pickle
import numpy as np
from elmoformanylangs import Embedder
import keras
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Bidirectional, TimeDistributed
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D, Input, add
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import classification_report as classrep
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Training Data Processing

In [ ]:
# get the training dataset directory
directory_1 = "gdrive/My Drive/GE Capstone/Data/i2b2/training-PHI-Gold-Set1"
directory_2 = "gdrive/My Drive/GE Capstone/Data/i2b2/training-PHI-Gold-Set2"

In [ ]:
# check the number of files in these two datasets
print("The training-PHI-Gold-Set1 has {} files. ".format(len(os.listdir(directory_1))))
print("The training-PHI-Gold-Set2 has {} files. ".format(len(os.listdir(directory_2))))

The training-PHI-Gold-Set1 has 621 files. 
The training-PHI-Gold-Set2 has 169 files. 


In [ ]:
directory = [directory_1, directory_2]

In [ ]:
list_of_sentences = [] # a list of list, the inner list is a list of tokens of a sentence
list_of_tags = [] # a list of tags indicating 0 or 1

In [ ]:
# get word tokens of all of the sentences and labels
# label: 0 --> non-PHI tags, 1 --> PHI tags
for dir in directory:
  for filename in os.listdir(dir):
    f = os.path.join(dir, filename)
    if os.path.isfile(f):
        with open(f) as fp:
          soup = BeautifulSoup(fp, features="xml")
          # get the origin text
          text = soup.find('TEXT').contents[0]

          # get all the tags
          tag_list_ground_truth = []
          tags = soup.find('TAGS')
          for item in tags.children:
            try:
              tag_word = item['text']
              tag_list_ground_truth .append(tag_word)
            except:
              pass

          sentence_ls = re.split('\n|\.', text)
          for sentence in sentence_ls:
            if len(sentence) != 0:
              tokens = word_tokenize(sentence.strip())
              list_of_sentences.append(tokens)

              # get the tags
              tag_list = []
              for token in tokens:
                is_found = False
                for tag in tag_list_ground_truth:
                  if tag.__contains__(token):
                    is_found = True
                    break

                if is_found == True:
                  tag_list.append(1)
                else:
                  tag_list.append(0)

              # add the tag_list of each sentence into the list_of_tags
              list_of_tags.append(tag_list)

          # check
          assert len(tag_list) == len(tokens)

In [ ]:
# remove the empty sentence or tags
list_of_sentences = [sen for sen in list_of_sentences if len(sen) != 0]
list_of_tags = [tag for tag in list_of_tags if len(tag) != 0]

In [ ]:
# checking
for i in range (len(list_of_sentences)):
  assert len(list_of_sentences[i]) == len(list_of_tags[i])

assert len(list_of_sentences) == len(list_of_tags)
print("There are totally {} sentences in the training dataset.".format(len(list_of_sentences)))

There are totally 86342 sentences in the training dataset.


In [ ]:
# check the length of each sentence
print(sorted([len(sen) for sen in list_of_sentences], reverse=True))
temp_max = sorted([len(sen) for sen in list_of_sentences], reverse=True)[0]

[306, 219, 135, 131, 123, 122, 122, 116, 113, 102, 99, 98, 97, 95, 94, 89, 88, 86, 86, 82, 81, 79, 79, 78, 77, 77, 74, 74, 73, 73, 73, 72, 72, 71, 70, 70, 70, 70, 70, 69, 69, 69, 69, 69, 68, 68, 68, 67, 67, 67, 67, 67, 66, 66, 66, 65, 65, 65, 64, 63, 63, 63, 62, 62, 62, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 60, 60, 60, 60, 60, 60, 60, 60, 60, 59, 59, 59, 59, 59, 59, 58, 58, 58, 58, 57, 57, 57, 57, 57, 57, 57, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 54, 54, 54, 54, 54, 54, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 51, 51, 51, 51, 51, 51, 51, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 49, 49, 49, 49, 49, 49, 49, 49, 49, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 45, 45, 4

In [ ]:
# # shorten the longest sentence
# longest_sentence = ""
# longest_tag = ""

# for i in range (len(list_of_sentences)):
#   if len(list_of_sentences[i]) == 306:
#     longest_sentence = list_of_sentences[i]
#     longest_tag = list_of_tags[i]
#     break

# list_of_sentences.remove(longest_sentence)
# list_of_tags.remove(longest_tag)

# print(len(list_of_sentences), len(list_of_tags))

# # add the halved sentences
# list_of_sentences.append(longest_sentence[:temp_max//2])
# list_of_sentences.append(longest_sentence[temp_max//2:])

# list_of_tags.append(longest_tag[:temp_max//2])
# list_of_tags.append(longest_tag[temp_max//2:])

# print(len(list_of_sentences), len(list_of_tags))

In [ ]:
# Tokenize the sentences and transform into integer representation
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list_of_sentences)
list_of_tokens_int = tokenizer.texts_to_sequences(list_of_sentences)

In [ ]:
# get the word embeddings of the training dataset
max_len = max(len(s) for s in list_of_tokens_int)
X_train = pad_sequences(list_of_tokens_int, maxlen=max_len, padding='post')

# get the labels of the training dataset
y_train = pad_sequences(list_of_tags, maxlen=max_len, padding='post')

# look at the shape
print("Shape of X_train: {}".format(X_train.shape))
print("Shape of y_train: {}".format(y_train.shape))

Shape of X_train: (86342, 306)
Shape of y_train: (86342, 306)


In [ ]:
# Create the Bi-directional LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_len))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 306, 50)           1429350   
                                                                 
 bidirectional (Bidirection  (None, 306, 128)          58880     
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 306, 1)            129       
 ributed)                                                        
                                                                 
Total params: 1488359 (5.68 MB)
Trainable params: 1488359 (5.68 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint(filepath='models/best_model_lstm.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
# Train the model
batch_size = 128
epochs = 20
history = model.fit(X_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=1, callbacks=callbacks)

Epoch 1/20
540/540 [==============================] - 46s 73ms/step - loss: 0.0203 - accuracy: 0.9970 - val_loss: 0.0052 - val_accuracy: 0.9981
Epoch 2/20
  4/540 [..............................] - ETA: 12s - loss: 0.0054 - accuracy: 0.9978

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


540/540 [==============================] - 18s 33ms/step - loss: 0.0045 - accuracy: 0.9981 - val_loss: 0.0045 - val_accuracy: 0.9982
Epoch 3/20
540/540 [==============================] - 16s 29ms/step - loss: 0.0036 - accuracy: 0.9985 - val_loss: 0.0043 - val_accuracy: 0.9983
Epoch 4/20
540/540 [==============================] - 15s 28ms/step - loss: 0.0033 - accuracy: 0.9986 - val_loss: 0.0044 - val_accuracy: 0.9983
Epoch 5/20
540/540 [==============================] - 15s 28ms/step - loss: 0.0032 - accuracy: 0.9987 - val_loss: 0.0044 - val_accuracy: 0.9982
Epoch 6/20
540/540 [==============================] - 15s 27ms/step - loss: 0.0031 - accuracy: 0.9987 - val_loss: 0.0046 - val_accuracy: 0.9983
Epoch 7/20
540/540 [==============================] - 15s 28ms/step - loss: 0.0030 - accuracy: 0.9988 - val_loss: 0.0046 - val_accuracy: 0.9982
Epoch 8/20
540/540 [==============================] - 15s 28ms/step - loss: 0.0028 - accuracy: 0.9988 - val_loss: 0.0047 - val_accuracy: 0.9982


## Test Data Processing

In [ ]:
test_dir = "gdrive/My Drive/GE Capstone/Data/i2b2/testing-PHI-Gold-fixed"
# check the number of files in these two datasets
print("The testing-PHI-Gold-fixed has {} files. ".format(len(os.listdir(test_dir))))

The testing-PHI-Gold-fixed has 514 files. 


In [ ]:
list_of_sentences_test = [] # a list of list, the inner list is a list of tokens of a sentence
list_of_tags_test = [] # a list of tags indicating 0 or 1

In [ ]:
# get word tokens of all of the sentences and labels
# label: 0 --> non-PHI tags, 1 --> PHI tags
for filename in os.listdir(test_dir):
  f = os.path.join(test_dir, filename)
  if os.path.isfile(f):
      with open(f) as fp:
        soup = BeautifulSoup(fp, features="xml")
        # get the origin text
        text = soup.find('TEXT').contents[0]

        # get all the tags
        tag_list_ground_truth = []
        tags = soup.find('TAGS')
        for item in tags.children:
          try:
            tag_word = item['text']
            tag_list_ground_truth .append(tag_word)
          except:
            pass

        sentence_ls = re.split('\n|\.', text)
        for sentence in sentence_ls:
          if len(sentence) != 0:
            tokens = word_tokenize(sentence.strip())
            list_of_sentences_test.append(tokens)

            # get the tags
            tag_list = []
            for token in tokens:
              is_found = False
              for tag in tag_list_ground_truth:
                if tag.__contains__(token):
                  is_found = True
                  break

              if is_found == True:
                tag_list.append(1)
              else:
                tag_list.append(0)

            # add the tag_list of each sentence into the list_of_tags_test
            list_of_tags_test.append(tag_list)

        # check
        assert len(tag_list) == len(tokens)

In [ ]:
# remove the empty sentences or tags
list_of_sentences_test = [sen for sen in list_of_sentences_test if len(sen) != 0]
list_of_tags_test = [tag for tag in list_of_tags_test if len(tag) != 0]

In [ ]:
# checking
for i in range (len(list_of_sentences_test)):
  assert len(list_of_sentences_test[i]) == len(list_of_tags_test[i])

assert len(list_of_sentences_test) == len(list_of_tags_test)
print("There are totally {} sentences in the test dataset.".format(len(list_of_sentences_test)))

There are totally 55447 sentences in the test dataset.


In [ ]:
# check the length of each sentence
print(sorted([len(sen) for sen in list_of_sentences_test], reverse=True))
# temp_max = sorted([len(sen) for sen in list_of_sentences], reverse=True)[0]

[274, 195, 181, 146, 140, 134, 118, 111, 107, 104, 98, 98, 96, 95, 94, 88, 87, 85, 81, 79, 79, 78, 78, 76, 76, 75, 75, 74, 73, 72, 71, 71, 70, 70, 70, 69, 69, 67, 67, 67, 65, 65, 64, 63, 63, 63, 63, 62, 62, 61, 61, 60, 60, 60, 60, 60, 60, 60, 58, 58, 57, 57, 56, 56, 56, 56, 56, 56, 56, 55, 55, 55, 55, 55, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 53, 53, 53, 53, 53, 53, 53, 53, 53, 52, 52, 52, 52, 52, 52, 52, 52, 52, 51, 51, 51, 51, 51, 51, 51, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 49, 49, 49, 49, 49, 49, 49, 49, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4

In [ ]:
# Function to replace unknown words with a placeholder
def texts_to_sequences_custom(tokenizer, texts, placeholder_token="<UNK>"):
    word_index = tokenizer.word_index
    unknown_token_index = word_index.get(placeholder_token, None)

    # If the placeholder token does not exist, add it to the word_index
    if unknown_token_index is None:
        unknown_token_index = len(word_index) + 1
        word_index[placeholder_token] = unknown_token_index

    sequences = []
    for text in texts:
        text_sequence = []
        for word in text:
            word_sequence = word_index.get(word.lower(), unknown_token_index)
            text_sequence.append(word_sequence)
        sequences.append(text_sequence)

    return sequences

In [ ]:
# get the integer representation of test dataset
list_of_tokens_test_int = texts_to_sequences_custom(tokenizer, list_of_sentences_test)

In [ ]:
# get the word embeddings of the test dataset
# max_len = max(len(s) for s in list_of_tokens_test_int)
X_test = pad_sequences(list_of_tokens_test_int, maxlen=max_len, padding='post')

# get the labels of the training dataset
y_test = pad_sequences(list_of_tags_test, maxlen=max_len, padding='post')

# look at the shape
print("Shape of X_test: {}".format(X_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_test: (55447, 306)
Shape of y_test: (55447, 306)


In [ ]:
# make predictions
y_test_pred = model.predict(X_test)
print("Shape of raw y_test_pred: {}".format(y_test_pred.shape))

dim1, dim2 = y_test_pred.shape[0], y_test_pred.shape[1]
y_test_pred = (y_test_pred > 0.5).astype(int).reshape(dim1, dim2)
print("Shape of y_test_pred: {}".format(y_test_pred.shape))

1733/1733 [==============================] - 14s 8ms/step
Shape of raw y_test_pred: (55447, 306, 1)
Shape of y_test_pred: (55447, 306)


In [ ]:
max([len(sen) for sen in list_of_sentences_test])

274

## Model Evaluation

In [ ]:
# for each sentence, we need to remove the paddings for evaluation
y_test_unpadding = []
y_test_pred_unpadding = []
for i in range (len(list_of_sentences_test)):
  length = len(list_of_sentences_test[i])
  y_test_unpadding.append(y_test[i, :][:length])
  y_test_pred_unpadding.append(y_test_pred[i, :][:length])

In [ ]:
assert len(y_test_unpadding) == len(y_test_pred_unpadding)
print(len(y_test_unpadding))

55447


In [ ]:
# flatten
y_test_flatten = [element for sublist in y_test_unpadding for element in sublist]
y_test_pred_flatten = [element for sublist in y_test_pred_unpadding for element in sublist]
print(len(y_test_flatten), len(y_test_pred_flatten))

370037 370037


In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test_flatten, y_test_pred_flatten)
precision = precision_score(y_test_flatten, y_test_pred_flatten)
recall = recall_score(y_test_flatten, y_test_pred_flatten)
f1 = f1_score(y_test_flatten, y_test_pred_flatten)

print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test F1-Score: {f1}")

Test Accuracy: 0.910103584236171
Test Precision: 0.7773015022523864
Test Recall: 0.6398908710710318
Test F1-Score: 0.7019345358099692
